# lDE15 Image Processing Notebook

See './Image_Analysis/Examples/1_Core_Image_Processing.ipynb' for a documented example of notebook usage.

#### Imports

In [ ]:
import paulssonlab.deaton.trenchripper.trenchripper as tr

import warnings

warnings.filterwarnings(action="once")

import matplotlib

matplotlib.rcParams["figure.figsize"] = [20, 10]

In [ ]:
# addition of active memory manager
import dask
dask.config.set({'distributed.scheduler.active-memory-manager.start': True});
dask.config.set({'distributed.scheduler.worker-ttl': "5m"});
dask.config.set({'distributed.scheduler.allowed-failures': 100});

dask_wd = "/home/de64/scratch/de64/dask"

# Part 0: Flat Field Generation

In [ ]:
import tifffile
from matplotlib import pyplot as plt

### Placeholder Dark Image

In [ ]:
import numpy as np

dark_img = np.zeros((2960, 2960),dtype="uint16")+100

In [ ]:
tifffile.imsave("/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/fake_dark_img.tiff",data=dark_img)

### Cy5-Penta

In [ ]:
flatfieldpath = "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/Cy5-Penta_20x_Ph2.nd2"
outputpath = "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/Cy5-Penta_20x_Ph2_flatfield.tiff"
tr.generate_flatfield(flatfieldpath,outputpath)

In [ ]:
plt.imshow(tifffile.imread("/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/Cy5-Penta_20x_Ph2_flatfield.tiff"))

### Cy7-Penta

In [ ]:
flatfieldpath = "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/Cy7-Penta_20x_Ph2.nd2"
outputpath = "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/Cy7-Penta_20x_Ph2_flatfield.tiff"
generate_flatfield(flatfieldpath,outputpath)

In [ ]:
plt.imshow(tifffile.imread("/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/Cy7-Penta_20x_Ph2_flatfield.tiff"))

### GFP-Penta

In [ ]:
flatfieldpath = "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/GFP-Penta_20x_Ph2.nd2"
outputpath = "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/GFP-Penta_20x_Ph2_flatfield.tiff"
generate_flatfield(flatfieldpath,outputpath)

In [ ]:
plt.imshow(tifffile.imread("/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/GFP-Penta_20x_Ph2_flatfield.tiff"))

### MCherry

In [ ]:
flatfieldpath = "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/MCHERRY_20x_Ph2.nd2"
outputpath = "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/MCHERRY_20x_Ph2_flatfield.tiff"
generate_flatfield(flatfieldpath,outputpath)

In [ ]:
plt.imshow(tifffile.imread("/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/MCHERRY_20x_Ph2_flatfield.tiff"))

### RFP-Penta

In [ ]:
flatfieldpath = "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/RFP-Penta_20x_Ph2.nd2"
outputpath = "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/RFP-Penta_20x_Ph2_flatfield.tiff"
generate_flatfield(flatfieldpath,outputpath)

In [ ]:
import tifffile 
tifffile.imread("/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/RFP-Penta_20x_Ph2_flatfield.tiff")

In [ ]:
plt.imshow(tifffile.imread("/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/RFP-Penta_20x_Ph2_flatfield.tiff"))

# Part 1: GFP

#### Specify Paths

In [ ]:
headpath = "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/GFP/"
nd2file = "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/experiment.nd2"

## Extract to hdf5 files

#### Start Dask Workers

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="00:30:00",
    local=False,
    n_workers=100,
    memory="2GB",
    working_directory=headpath + "/dask",
)
dask_controller.startdask()

In [ ]:
dask_controller.daskclient

##### Perform Extraction

In [ ]:
hdf5_extractor = tr.ndextract.hdf5_fov_extractor(
    nd2file,
    headpath,
    tpts_per_file=50,
    ignore_fovmetadata=False,
    nd2reader_override={"z_levels": [], "z_coordinates": []},
)

##### Extraction Parameters

In [ ]:
hdf5_extractor.inter_set_params()

#### List of paths to flat fields
 - /home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/RFP-Penta_20x_Ph2_flatfield.tiff
 - /home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/GFP-Penta_20x_Ph2_flatfield.tiff
 - /home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/flat_field/fake_dark_img.tiff

In [ ]:
hdf5_extractor.inter_set_flatfieldpaths()

##### Begin Extraction 

In [ ]:
hdf5_extractor.extract(dask_controller)

##### Shutdown Dask

In [ ]:
dask_controller.shutdown()

## Kymographs

### Test Parameters



##### Initialize the interactive kymograph class

In [ ]:
interactive_kymograph = tr.kymograph_interactive(headpath)

In [ ]:
interactive_kymograph.import_hdf5_interactive()

##### Tune "trench-row" detection hyperparameters

In [ ]:
interactive_kymograph.preview_y_precentiles_interactive()

##### Tune "trench-row" cropping hyperparameters

In [ ]:
interactive_kymograph.preview_y_precentiles_consensus_interactive()

In [ ]:
interactive_kymograph.preview_y_crop_interactive()

##### Tune trench detection hyperparameters

In [ ]:
interactive_kymograph.preview_x_percentiles_interactive()

##### Tune trench cropping hyperparameters

In [ ]:
interactive_kymograph.preview_kymographs_interactive()

##### Export and save hyperparameters

In [ ]:
interactive_kymograph.process_results()

In [ ]:
interactive_kymograph.write_param_file()

### Generate Kymograph

##### Start Dask Workers

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="04:00:00",
    local=False,
    n_workers=200,
    memory="2GB",
    working_directory=headpath + "/dask",
)
dask_controller.startdask()

In [ ]:
dask_controller.daskclient

##### Perform Kymograph Cropping

In [ ]:
kymoclust = tr.kymograph.kymograph_cluster(
    headpath=headpath, trenches_per_file=2000, paramfile=True
)

##### Begin Kymograph Cropping 

In [ ]:
kymoclust.generate_kymographs(dask_controller)

##### Post-process Images

In [ ]:
kymoclust.post_process(dask_controller)

##### Check kymograph statistics

In [ ]:
kymoclust.kymo_report()

##### Shutdown Dask

In [ ]:
dask_controller.shutdown()

## Fluorescence Segmentation

Now that you have copped your data into kymographs, we will now perform segmentation/cell detection <br>
on your kymographs. Currently, this pipeline only supports segmentation of fluorescence images; however, <br>
segmentation of transmitted light imaging techniques is in development.

The output of this step will be a set of `segmentation_[File #].hdf5` files stored in `headpath/fluorsegmentation`.<br>
The image data stored in these files takes the exact same form as the kymograph data, `(K,T,Y,X)` arrays <br>
where K is the trench index, T is time, and Y,X are the crop dimensions. These arrays are accessible using <br>
keys of the form `"[Trench Row Number]"`.

Since no metadata is generated by this step, it is possible to use another segmentation algorithm on the kymograph <br>
data. The output of segmentation must be split into `segmentation_[File #].hdf5` files, where `[File #]` agrees with the<br>
corresponding `kymograph_[File #].hdf5` file. Additionally, the `(K,T,Y,X)` arrays must be of the same shape as the <br>
kymograph arrays and accessible at the corresponding `"[Trench Row Number]"` key. These files must be placed into <br>
their own folder at `headpath/foldername`. This folder may then be used in later steps.

### Test Parameters

##### Initialize the interactive segmentation class

As a first step, initialize the `tr.fluo_segmentation_interactive` class that will be handling all steps of generating a segmentation. 

In [ ]:
interactive_segmentation = tr.fluo_segmentation_interactive(headpath)

##### Choose channel to segment on

In [ ]:
interactive_segmentation.choose_seg_channel_inter()

#### Import data

Fill in 

You will need to tune the following `args` and `kwargs` (in order):

**fov_idx (int)** :

**n_trenches (int)** :

**t_range (tuple)** :

**t_subsample_step (int)** :

In [ ]:
interactive_segmentation.import_array_inter()

##### Process data

In [ ]:
interactive_segmentation.plot_processed_inter()

#### Determine Cell Mask Envelope

Fill in.

You will need to tune the following `args` and `kwargs` (in order):

**cell_mask_method (str)** : Thresholding method, can be a local or global Otsu threshold.

**cell_otsu_scaling (float)** : Scaling factor applied to determined threshold.

**local_otsu_r (int)** : Radius of thresholding kernel used in the local otsu thresholding.

In [ ]:
interactive_segmentation.plot_cell_mask_inter()

In [ ]:
interactive_segmentation.plot_eig_mask_inter()

In [ ]:
interactive_segmentation.plot_dist_mask_inter()

In [ ]:
interactive_segmentation.plot_marker_mask_inter()

In [ ]:
interactive_segmentation.process_results()

In [ ]:
interactive_segmentation.write_param_file()

### Generate Segmentation

#### Start Dask Workers

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="01:00:00",
    local=False,
    n_workers=200,
    memory="2GB",
    working_directory=headpath + "/dask",
)
dask_controller.startdask()

In [ ]:
dask_controller.displaydashboard()

In [ ]:
segment = tr.segment.fluo_segmentation_cluster(headpath, paramfile=True)

In [ ]:
segment.dask_segment(dask_controller)

#### Stop Dask Workers

In [ ]:
dask_controller.shutdown()

## Region Properties (No Lineage)

In [ ]:
analyzer = tr.regionprops_extractor(headpath, "fluorsegmentation", intensity_channel_list=["RFP-Penta", "GFP-Penta"], include_background = True)

In [ ]:
analyzer.export_all_data(n_workers=50)

# Part 2: Barcodes

#### Specify Paths

In [ ]:
headpath = "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/"
hdf5inputpath = "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/run/"

## Extract to hdf5 files

#### Start Dask Workers

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="04:00:00",
    local=False,
    n_workers=40,
    memory="4GB",
    working_directory=headpath + "/dask",
)
dask_controller.startdask()

In [ ]:
dask_controller.daskclient

##### Perform Extraction

In [ ]:
hdf5_extractor = tr.marlin_extractor(hdf5inputpath, headpath, metaparsestr='metadata_{timepoint:d}.hdf5')

##### Extraction Parameters

In [ ]:
hdf5_extractor.inter_set_params()

##### Begin Extraction 

In [ ]:
hdf5_extractor.extract(dask_controller)

##### Shutdown Dask

In [ ]:
dask_controller.daskclient.restart()

In [ ]:
dask_controller.shutdown()

## Kymographs

### Test Parameters



##### Initialize the interactive kymograph class

In [ ]:
interactive_kymograph = tr.kymograph_interactive(headpath)

In [ ]:
interactive_kymograph.import_hdf5_interactive()

##### Tune "trench-row" detection hyperparameters

In [ ]:
interactive_kymograph.preview_y_precentiles_interactive()

##### Tune "trench-row" cropping hyperparameters

In [ ]:
interactive_kymograph.preview_y_precentiles_consensus_interactive()

In [ ]:
interactive_kymograph.preview_y_crop_interactive()

##### Tune trench detection hyperparameters

In [ ]:
interactive_kymograph.preview_x_percentiles_interactive()

##### Tune trench cropping hyperparameters

In [ ]:
interactive_kymograph.preview_kymographs_interactive()

##### Export and save hyperparameters

In [ ]:
interactive_kymograph.process_results()

In [ ]:
interactive_kymograph.write_param_file()

### Generate Kymograph

##### Start Dask Workers

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="04:00:00",
    local=False,
    n_workers=100,
    memory="4GB",
    working_directory=headpath + "/dask",
)
dask_controller.startdask()

In [ ]:
dask_controller.daskclient

##### Perform Kymograph Cropping

In [ ]:
kymoclust = tr.kymograph.kymograph_cluster(
    headpath=headpath, trenches_per_file=2000, paramfile=True
)

##### Begin Kymograph Cropping

In [ ]:
kymoclust.generate_kymographs(dask_controller)

##### Post-process Images

In [ ]:
kymoclust.post_process(dask_controller)

##### Check kymograph statistics

In [ ]:
kymoclust.kymo_report()

##### Shutdown Dask

In [ ]:
dask_controller.daskclient.restart()

In [ ]:
dask_controller.shutdown()

## FISH Analysis

In [ ]:
headpath = "/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/barcodes"

##### Start Dask Workers

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="02:00:00",
    local=False,
    n_workers=50,
    memory="16GB",
    working_directory=headpath + "/dask",
)
dask_controller.startdask()

In [ ]:
dask_controller.displaydashboard()

#### Get Barcode Signal (Percentile Function)

In [ ]:
import numpy as np

tr.get_all_image_measurements(
    dask_controller,
    headpath,
    headpath + "/percentiles",
    ["RFP","Cy5","Cy7"],
    "98th Percentile",
    np.percentile,
    98)

In [ ]:
fish_test = tr.fish_analysis(headpath,"../../../nanopore/dev_notebooks/2021-09-16_snakemake_lDE15_v2/lDE15_final_df.tsv",hamming_thr=1,remove_bit_list=[],channel_names=["RFP 98th Percentile","Cy5 98th Percentile","Cy7 98th Percentile"])

In [ ]:
fish_test.plot_signal_threshold_inter()

In [ ]:
fish_test.get_bit_thresholds()

In [ ]:
fish_test.bit_threshold_list = [1300,
 1200,
 2000,
 1800,
 2000,
 1800,
 2000,
 2000,
 2000,
 2000,
 4416,
 5443,
 5011,
 4314,
 4793,
 3661,
 3841,
 3714,
 3647,
 6282,
 1000,
 1000,
 750,
 750,
 1000,
 700,
 800,
 800,
 700,
 700]

In [ ]:
fish_test.plot_bit_threshold_inter()

In [ ]:
fish_test.export_bit_thresholds()

In [ ]:
fish_test.output_barcode_df("/home/de64/scratch/de64/sync_folder/2021-10-21_lDE15_Final_1/barcodes/hdf5/barcode_df.h5py")